In [9]:
import ee
from IPython.display import Image
import ee.mapclient

# Initialize the Earth Engine module.
ee.Initialize()

# Display a thumbnail of a sample image asset.
Image(url=ee.Image('CGIAR/SRTM90_V4').getThumbUrl({'min': 0, 'max': 3000}))

In [10]:
# Download example

# Get a download URL for an image.
image = ee.Image('CGIAR/SRTM90_V4')
path = image.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
print(path)

https://earthengine.googleapis.com/api/download?docid=bb00bf1172379e96ae3436ef1c511da4&token=59f07995d3830144a29ef889f03a1be6


In [11]:
# Display an image given its ID.
ee.mapclient.addToMap(image, {'min': 0, 'max': 3000})

RuntimeError: main thread is not in main loop

In [5]:
#Display portions of an image with different dynamic ranges.
#The land areas are displayed normally, but the water areas
#are streched to show more details.

import datetime
import ee
import ee.mapclient

ee.Initialize()
ee.mapclient.centerMap(-95.738, 18.453, 9)

# Filter the LE7 collection to a single date.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2002, 11, 8),
                          datetime.datetime(2002, 11, 9)))
image = collection.mosaic().select('B3', 'B2', 'B1')

# Display the image normally.
ee.mapclient.addToMap(image, {'gain': '1.6, 1.4, 1.1'}, 'Land')

# Add and stretch the water.  Once where the elevation is masked,
# and again where the elevation is zero.
elev = ee.Image('CGIAR/SRTM90_V4')
mask1 = elev.mask().eq(0).And(image.mask())
mask2 = elev.eq(0).And(image.mask())
ee.mapclient.addToMap(
    image.mask(mask1), {'gain': 6.0, 'bias': -200}, 'Water: Masked')
ee.mapclient.addToMap(
    image.mask(mask2), {'gain': 6.0, 'bias': -200}, 'Water: Elev 0')

In [6]:
# Compute hillshade from elevation.

import math
import ee
import ee.mapclient

ee.Initialize()
ee.mapclient.centerMap(-121.767, 46.852, 11)


def Radians(img):
    return img.toFloat().multiply(math.pi).divide(180)


def Hillshade(az, ze, slope, aspect):
    """Compute hillshade for the given illumination az, el."""
    azimuth = Radians(ee.Image(az))
    zenith = Radians(ee.Image(ze))
    # Hillshade = cos(Azimuth - Aspect) * sin(Slope) * sin(Zenith) +
    #     cos(Zenith) * cos(Slope)
    return (azimuth.subtract(aspect).cos()
          .multiply(slope.sin())
          .multiply(zenith.sin())
          .add(
              zenith.cos().multiply(slope.cos())))

terrain = ee.Algorithms.Terrain(ee.Image('CGIAR/SRTM90_V4'))
slope_img = Radians(terrain.select('slope'))
aspect_img = Radians(terrain.select('aspect'))

# Add 1 hillshade at az=0, el=60.
ee.mapclient.addToMap(Hillshade(0, 60, slope_img, aspect_img))

In [7]:
# There are many fine places to look here is one.  Comment
# this out if you want to twiddle knobs while panning around.
#ee.mapclient.centerMap(-61.61625, -11.64273, 14)

# Grab a sample L7 image and pull out the RGB and pan bands
# in the range (0, 1).  (The range of the pan band values was
# chosen to roughly match the other bands.)
image1 = ee.Image('LANDSAT/LE07/C01/T1/LE07_230068_19990815')

rgb = image1.select('B3', 'B2', 'B1').unitScale(0, 255)
gray = image1.select('B8').unitScale(0, 155)

# Convert to HSV, swap in the pan band, and convert back to RGB.
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat(huesat, gray).hsvToRgb()

# Display before and after layers using the same vis parameters.
visparams = {'min': [.15, .15, .25], 'max': [1, .9, .9], 'gamma': 1.6}
ee.mapclient.addToMap(rgb, visparams, 'Orignal')
ee.mapclient.addToMap(upres, visparams, 'Pansharpened')

In [8]:
ee.mapclient.centerMap(-113.41842, 40.055489, 6)

# Force projection of 500 meters/pixel, which is the native MODIS resolution.
VECTORIZATION_SCALE = 500

image1 = ee.Image('MODIS/051/MCD12Q1/2001_01_01')
image2 = image1.select(['Land_Cover_Type_1'])
image3 = image2.reproject('EPSG:4326', None, 500)
image4 = image3.focal_mode()
image5 = image4.focal_max(3).focal_min(5).focal_max(3)
image6 = image5.reproject('EPSG:4326', None, 500)

PALETTE = [
    'aec3d4',  # water
    '152106', '225129', '369b47', '30eb5b', '387242',  # forest
    '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass, savannah
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f'   # tundra
    ]

vis_params = {'min': 0, 'max': 17, 'palette': PALETTE}

ee.mapclient.addToMap(image2, vis_params, 'IGBP classification')
ee.mapclient.addToMap(image3, vis_params, 'Reprojected')
ee.mapclient.addToMap(image4, vis_params, 'Mode')
ee.mapclient.addToMap(image5, vis_params, 'Smooth')
ee.mapclient.addToMap(image6, vis_params, 'Smooth')

Exception in thread Thread-135:
Traceback (most recent call last):
  File "F:\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "F:\anaconda\lib\site-packages\ee\mapclient.py", line 459, in run
    callback(image)
  File "F:\anaconda\lib\site-packages\ee\mapclient.py", line 231, in AddTile
    self.tktiles[key].paste(newtile)
  File "F:\anaconda\lib\site-packages\PIL\ImageTk.py", line 181, in paste
    tk.call("PyImagingPhoto", self.__photo, block.id)
RuntimeError: main thread is not in main loop

Exception in thread Thread-133:
Traceback (most recent call last):
  File "F:\anaconda\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "F:\anaconda\lib\site-packages\ee\mapclient.py", line 459, in run
    callback(image)
  File "F:\anaconda\lib\site-packages\ee\mapclient.py", line 231, in AddTile
    self.tktiles[key].paste(newtile)
  File "F:\anaconda\lib\site-packages\PIL\ImageTk.py", line 181, in paste
    tk.call("PyImagingPhoto", self.

In [ ]:
ee.mapclient.centerMap(-113.41842, 40.055489, 6)

elev = ee.Image('CGIAR/SRTM90_V4')
cover = ee.Image('MODIS/051/MCD12Q1/2001_01_01').select('Land_Cover_Type_1')
blank = ee.Image(0)

# Where (1 <= cover <= 4) and (elev > 1000), set the output to 1.
output = blank.where(
    cover.lte(4).And(cover.gte(1)).And(elev.gt(1000)),
    1)

# Output contains 0s and 1s.  Mask it with itself to get rid of the 0s.
result = output.mask(output)

ee.mapclient.addToMap(result, {'palette': '00AA00'})

In [ ]:
import datetime

ee.mapclient.centerMap(-110, 40, 5)

# Create a Landsat 7, median-pixel composite for Spring of 2000.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2000, 4, 1),
                          datetime.datetime(2000, 7, 1)))
image1 = collection.median()

# Clip to the output image to the California state boundary.
fc = (ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8')
      .filter(ee.Filter().eq('Name', 'California')))
image2 = image1.clipToCollection(fc)

# Select the red, green and blue bands.
image = image2.select('B3', 'B2', 'B1')
ee.mapclient.addToMap(image, {'gain': [1.4, 1.4, 1.1]})

In [ ]:
ee.mapclient.centerMap(-110, 40, 5)

# Filter to only include images within the colorado and utah boundaries.
polygon = ee.Geometry.Polygon([[
    [-109.05, 37.0], [-102.05, 37.0], [-102.05, 41.0],   # colorado
    [-109.05, 41.0], [-111.05, 41.0], [-111.05, 42.0],   # utah
    [-114.05, 42.0], [-114.05, 37.0], [-109.05, 37.0]]])

# Create a Landsat 7 composite for Spring of 2000, and filter by
# the bounds of the FeatureCollection.
collection = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
              .filterDate(datetime.datetime(2000, 4, 1),
                          datetime.datetime(2000, 7, 1))
              .filterBounds(polygon))

# Select the median pixel.
image1 = collection.median()

# Select the red, green and blue bands.
image = image1.select('B3', 'B2', 'B1')
ee.mapclient.addToMap(image, {'gain': [1.4, 1.4, 1.1]})